# Monad

## Outline

* Incentive for Monad

* Monads and "do" notation

* Monad examples

* Monad laws

In this lesson, we will learn about the Monad type classe and how you can use it.

## Incentive for Monad

We covered until now two cases with functor and applicative. If you have data in a `Maybe` cotext and the following functions:
```haskell
f1 :: a -> b
f2 :: Maybe (a -> b) 
```
then you can use the `<$>` and `<*>` operators to apply these two functions to data of type `Maybe` and get data of the same type returned. This works for the `Maybe` type because it has an instance of Functor and Applicative. You can also define your custom type and make it an instance of Functor and Applicative.

To describe it in a more general way: 

- we use Functors to solve the problem of mapping a function over a value inside a context
- then we use Applicative to solve the problem of what happens if the function is also inside the context. 

And now, we have a new problem: What if we have a value with a context `(m a)` and a funciton that takes a contextless value and returns a value with a context `(a -> m b)`? Let's take an example for the Maybe type that provides the context for missing values:
```haskell
f3 :: a -> Maybe b
```

Here is an example with the the `Maybe` type and the `fromJust` and `isJust` functions that help us to solve this problem.

In [ ]:
import Data.Maybe (fromJust, isJust)
import qualified Data.Map as Map

bookIDs :: Map.Map Int Int
bookIDs = Map.fromList $ zip [1,2,3] [2869, 7435, 1423]

bookPrices :: Map.Map Int String
bookPrices = Map.fromList $ zip [2869, 7435, 1423] ["15EUR","20USD","18JPY"]

getPriceFromID :: Int -> Maybe String
getPriceFromID n 
    | isJust id = Map.lookup (fromJust id) bookPrices
    | otherwise = Nothing
    where id = Map.lookup n bookIDs

print $ getPriceFromID 1

Let's now see how we can solve this problem with the Monad type class. The definition of Monad is the following:
```haskell
class Applicative m => Monad m where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
```

We see that Applicative is a superclass of Monad. The minimal complete definition requires just the bind operator `>>=`. Because the Maybe type has also an instance of Monad let's see how we can use the bind operator to rewrite the previous code example. We use also the `flip` function that reverses the order of input parameters for a function.

In [ ]:
getPriceFromIDBind :: Int -> Maybe String
getPriceFromIDBind n = id >>= flip Map.lookup bookPrices
    where id = Map.lookup n bookIDs

print $ getPriceFromIDBind 1

We get the same result as before and the function is two lines shorter. As in the Applicative lesson also here we state that the main reason why to use this operator is that it's easier to read the code because it's generic. The same symbol serves the same purpose for all Monads. 

## Monads and "do" notation

Let's check out now the other to operators from the Monad type class. They are called the sequencing operator `>>` and the `return` operator. We notice that `return` looks the same as `pure` from the Applicative type class. In fact it does the same thing: puts a variavle into a context. The reason why they both exist is that the Monad type class predates the Applicative type class and when the later was developed `pure` was added to it because you can then express the `<$>` operator by using only `pure` and `<*>`. 

More interesting is the `>>` operator. It seems that it takes two variables in a monad context and throws away the first. That makes sense when you want to chain commands together. Let's have a look of a function that runs in IO which has also an instance of Monad tpye class.

In [ ]:
chain :: String -> IO ()
chain msg = print "My message is:\n" >> print msg

chain "test"

We can now write a simple program that asks a user for his name and then prints back a greeting. We can use both the `>>` and the `>>=` operators.

In [ ]:
hello :: IO () 
hello = 
  print "What is your name?" >> 
  getLine >>= 
  (\name -> return ("Hello " ++ name)) >>= 
  print

hello

We can re-write the above program by using `do` notation which is basically syntactic sugar (a nicer way of writing the same thing) for the code above.

In [ ]:
hello' :: IO () 
hello' = do 
  print "What is your name?" 
  name <- getLine 
  print ("Hello " ++ name) 

If you compare the code you see that the `<-` arrow is used to assign a variable that comes in a context to a variable without that context. In our case the `getLine` function returns an `IO String` and the variable `name` is just of type `String`. The statements are then chained together with the `>>` and `>>=` operators.

As you may remember you can also declare some variables inside a `do` block with a `let`. Here is the code that shows the version using lambda functions and the Monad operators with the code that uses only the `do` notation. The functions `monadExample` and `monadExample'` basically do the same thing.

In [ ]:
monadExample :: IO Int 
monadExample = do 
  let a = return 1 :: IO Int 
  let func var = return (var + 1) :: IO Int 
  a >>= func

monadExample' :: IO Int 
monadExample' = 
  (\a -> 
    a >>= 
      (\var -> 
        return (var + 1) :: IO Int 
      ) 
  ) (return 1 :: IO Int)

main :: IO ()
main = do
    var1 <- monadExample
    var2 <- monadExample'
    print var1
    print var2

main

The `let` and `<-` statements are both used to assign variables but the diference is that the `<-` operator takes variables out of the context and stores them as a plain type. Where the `let` operator is used for classical declarations just as if you would declare a variable outside a function.

## Monad examples

Now that you have see how the IO Monad works you can use the same concept for other types that have an instance of Monad. Lets take for example a list.

In [ ]:
add1 :: Num a => [a] -> [a]
add1 myList = do
    element <- myList
    return $ element + 1

print $ add1 [1..3]

When you run the code above you see that even though you perform the (+ 1) operation on a signle element you get a list returned. You could generalize this function by using the Monad type constraint in the type signature and it would work for lists, maybe and other types that have an instance of Monad value, out of the box. 

In [ ]:
import qualified Data.Map as Map

add1Monad :: (Num a, Monad m) => m a -> m a
add1Monad var = do
    raw <- var
    return $ raw + 1

print $ add1Monad [1..3]
print $ add1Monad $ Just 1

Another example of Monads is list comprehension which is an elegant way to create a list by specifying the element conditions inside the list. Here is a example:

In [ ]:
import Control.Monad ( guard )

list1 = [0 .. 9 :: Int]

add1Even :: [Int] -> [Int]
add1Even myList = do
  n <- myList
  let k = n + 1
  guard(even k)
  return k
  
add1Even' :: [Int] -> [Int]
add1Even' myList = [k | n <- myList, let k = n + 1, even k] 

print $ add1Even list1
print $ add1Even' list1

The `guard` function is basically a filter that passes only values of `k` which are `True` fot the expression inside the brackets in `add1Even`. The function `add1Even` is just a de-sugarated version of the function `add1Even'`. In our last example lets create a custom type called `Writer` that holds two values of type `Int` and `String` that is just a string of the number. We make it an instance of Monad and then create a function that takes in three `Writer` variables and summs the numbers together and cocatenates the strings.

In [ ]:
import Control.Monad (ap, liftM)

data Writer a = Writer a String
    deriving Show

bindWriter :: Writer a -> (a -> Writer b) -> Writer b
bindWriter (Writer a xs) f =
  let
    Writer b ys = f a
  in
    Writer b $ xs ++ ys

instance Functor Writer where
    fmap = liftM

instance Applicative Writer where
    pure = return
    (<*>) = ap

instance Monad Writer where
    return a = Writer a ""
    (>>=) = bindWriter

tell :: String -> Writer ()
tell msg = Writer () msg

tellAndSum :: Writer Int -> Writer Int -> Writer Int -> Writer Int
tellAndSum x y z = do
    tell "Sumed numbers: " 
    sumInts x y z

sumInts :: Monad m => m Int -> m Int -> m Int -> m Int
sumInts mx my mz = do
    k <- mx
    l <- my
    m <- mz
    let s = k + l + m
    return s

print $ tellAndSum (Writer 1 "1 ") (Writer 2 "2 ") (Writer 3 "3")

In the above example we see that we have to also declare instances for Functor and Applicative because in the definition of Monad, Applicative is a superclass of Monad. And in the definition of Applicative, Functor is a superclass of Applicative. We do this with the functions that we import from the `Control.Monad` module.

Now that you know how to use the Monad type class you can check back again on the lesson **Basic IO** where we used Monad operators. More examples can be found also in the Homework section.

## Monad laws

Monad has 3 laws. They are as follows:

- Identity (left):<br>`return a >>= m = m a`

- Identity (right):<br>`m >>= return = m`

- Associativity:<br>`(a >>= b) >>= c = a >>= (\x -> b x >>= c)`

## Recap

In this lesson we've discussed:

- the motivation for introducing the Monad type 

- the Monad operators and "do" notation

- examples that show how to use the Monad type class

- laws that apply to the Monad type class